In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Performance of other models is not so good so trying InceptionNet this time to see if performance is better

In [ ]:
# imports
import timeit
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [ ]:
# Load data
base_dir="/kaggle/input/knee-osteoarthritis-dataset-with-severity/"
train_path=os.path.join(base_dir,'train')
test_path=os.path.join(base_dir,'test')
valid_path=os.path.join(base_dir,'val')

### Using InceptionNet this time to do fine tuning

In [ ]:
# important parameters for training model
model_name="InceptionResNetV2"
class_names=['Healthy','Doubtful','Minimal','Moderate','Severe']
target_size=(224,224)
epochs=100
batch_size=256
img_shape=(224,224,3)

In [ ]:
saved_model_path=os.path.join('models',f'model_{model_name}_ft.hdf5')

### Generating augmented and non augmented data generators

In [ ]:
"""
This function used to generate augmented images for the training data
"""
def return_Augmented_Images():
    return tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    horizontal_flip=True,
    brightness_range=[0.4,0.8],
    width_shift_range=[-50,0,50,30,-30],
    zoom_range=0.1,
    fill_mode="nearest",
    )


"""
This function used to generate augmented images for the training data
"""
def return_no_Augmentation_Images():
    return tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.resnet50.preprocess_input,)

In [ ]:
aug_datagen=return_Augmented_Images()
no_aug_datagen=return_no_Augmentation_Images()

### Training and validation data

In [ ]:
train_generator=aug_datagen.flow_from_directory(train_path,class_mode="categorical",target_size=target_size,shuffle=True)
valid_generator=no_aug_datagen.flow_from_directory(valid_path,class_mode="categorical",target_size=target_size,shuffle=False)

In [ ]:
y_train=train_generator.labels
y_val=valid_generator.labels

### To handle class imbalance, finding class weights to use them later

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
unique,counts=np.unique(y_train,return_counts=True)
print("Train data:",dict(zip(unique,counts)))
class_weights=compute_class_weight(class_weight="balanced",classes=np.unique(y_train),y=y_train)
train_class_weights=dict(enumerate(class_weights))
print(train_class_weights)

### Some callbacks

In [ ]:
#train data
classes=np.unique(y_train)
# using some callbacks
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0.01,patience=9,restore_best_weights=True)
plateau_LR=tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',factor=0.1,min_delta=0.01,min_lr=1e-10,patience=4,mode='auto')

### Using transfer learning approach to use pre trained model (Inception-Net-ResNet) to save training time

In [ ]:
# model
model=tf.keras.applications.inception_resnet_v2.InceptionResNetV2(input_shape=img_shape,include_top=False,weights="imagenet")

### Fine Tuning the model

In [ ]:
for layer in model.layers:
    layer.trainable=True
fine_tuned_model=tf.keras.models.Sequential([model,tf.keras.layers.GlobalAveragePooling2D(),tf.keras.layers.Dropout(0.25),
                                             tf.keras.layers.Dense(5,activation="softmax")])
fine_tuned_model.summary()

In [ ]:
fine_tuned_model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
start_time=timeit.default_timer()
history=fine_tuned_model.fit(train_generator,epochs=epochs,batch_size=batch_size,
                             callbacks=[early_stopping,plateau_LR],validation_data=valid_generator,
                             class_weight=train_class_weights,verbose=1)
stop_time=timeit.default_timer()
execution_time=(stop_time-start_time)/60
print(f"Model {model_name} fine tuning executed in {execution_time:.2f} minutes")

In [ ]:
fine_tuned_model.save(saved_model_path)

In [ ]:
"""
Function to plot loss vs accuracy for the model trained
"""
def plot_loss_accuracy(model, model_name):
    fig=plt.figure()
    plt.subplot(2,1,1)
    plt.plot(model.history.history["loss"])
    plt.plot(model.history.history["val_loss"])
    plt.title(f"{model_name} \n model loss")
    plt.ylabel("loss")
    plt.xlabel("epoch")
    plt.legend(["train","valid"],loc="upper right")
    plt.subplot(2,1,2)
    plt.plot(model.history.history["accuracy"])
    plt.plot(model.history.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train","valid"],loc="lower right")
    plt.tight_layout()
    

plot_loss_accuracy(fine_tuned_model, f"{model_name} Fine Tuning")


In [ ]:
"""
Function to plot confusion matrix
"""
def plot_confusion_matrix(ytrue,ypred,class_names,model_name):
    cm=confusion_matrix(y_true=ytrue.labels,y_pred=np.argmax(ypred,axis=1))
    cmn=cm.astype("float")/cm.sum(axis=1)[:,np.newaxis]
    plt.subplots(figsize=(6,5))
    sns.heatmap(cmn,annot=True,fmt=".2f",cmap="Purples",xticklabels=class_names,yticklabels=class_names,)
    plt.title(f"Confusion Matrix -{model_name}")
    plt.ylabel("Actual")
    plt.xlabel("Predicted")
    plt.show(block=False)

In [ ]:
"""
Functions to evaluate and predict the model performance
"""
def evaluate_model(data,name, model):
    score_model=model.evaluate(data,verbose=1)
    print(f"{name} loss: {score_model[0]:.2f}")
    print(f"{name} accuracy: {score_model[1]:.2f}")
    
def predict_model(data,model):
    predict_model=model.predict(data)
    return predict_model


evaluate_model(valid_generator, "Valid", fine_tuned_model)

In [ ]:
"""
Function to show various metrics
"""
def get_metrics(y_test,y_pred,model_name):
    acc=accuracy_score(y_test,y_pred)
    print(f"Accuracy Score - {model_name}: {acc:.2f}")
    print(classification_report(y_test, y_pred))
    
predict_model_ft = predict_model(valid_generator, fine_tuned_model)
get_metrics(valid_generator.labels,y_pred=np.argmax(predict_model_ft, axis=1),model_name=model_name)


In [ ]:
plot_confusion_matrix(valid_generator, predict_model_ft, class_names, f"{model_name} Fine Tuning")